ปฏิบัติการครั้งที่ 9 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

ให้เริ่มทำปฏิบัติการจาก colab notebook หรือไฟล์ *.ipynb ที่กำหนดให้ จากนั้นบันทึกไว้เป็นไฟล์ *.pdf แล้วส่งใน Assignments

ดาวน์โหลดข้อมูลรถยนต์ชนิดต่างใน link ข้างล่างนี้  
http://www.donlapark.cmustat.com/229351/data/elecequip.csv  

In [ ]:
# uploading the csv file to colab

!pip install wget

!python -m wget -o elecequip.csv http://www.donlapark.cmustat.com/229351/data/elecequip.csv

In [ ]:
# import module ที่ต้องใช้
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
# ฟังก์ชันที่เปลี่ยน string ให้เป็น datetime
def parser(dates):
  return datetime.strptime(dates, '%Y-%m')

# parse_dates ชื่อของคอลัมน์ที่จะเปลี่ยนให้เป็น datetime
# index_col ชื่อของคอลัมน์ที่จะให้เป็น index
# date_parser ฟังก์ชันที่เปลี่ยน string ให้เป็น datetime
data = pd.read_csv('elecequip.csv', parse_dates=['time'], 
                                        index_col='time',
                                        date_parser=parser)

print (data.head())

print (data.dtypes)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(data["value"]);

#Moving average

In [ ]:
#data['MA'] = data['value'].rolling(window=5,center=True).mean() 
data['MA'] = data['value'].rolling(window=12,center=True).mean().rolling(window=2).mean().shift(-1)

data.head(12)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(data['value'])
plt.plot(data['MA']);

#Classical decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib as mpl

mpl.rc("figure", figsize=(12,6))
result_add = seasonal_decompose(data['value'], model='additive')

result_add.plot();

In [ ]:
result_mul = seasonal_decompose(data['value'], model='multiplicative')

result_mul.plot();

###เรียกดูแต่ละส่วน

In [ ]:
print(result_add.trend)
print(result_add.seasonal)
print(result_add.resid)
print(result_add.observed)

#Exponential smoothing

In [ ]:
ts = data['value']

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

fit1 = SimpleExpSmoothing(ts).fit()

#ดูค่า l_0 (initial level) และ alpha (smoothing parameter)
print("l_0: ",fit1.model.params['initial_level']) #l_0
print("alpha: ", fit1.model.params['smoothing_level']) #alpha

### `fit.forecast` สำหรับการทำนายข้อมูลในอนาคต

In [ ]:
fcast1 = fit1.forecast(12).rename(r'$\alpha=%s$'%fit1.model.params['smoothing_level'])

fcast1

In [ ]:
ax = ts.plot(marker='o', color='black', figsize=(20,8))

fcast1.plot(marker='o', color='blue', ax=ax, legend=True);

###`fit.fittedvalues` เพื่อทำนายเทียบกับค่าที่แท้จริง

In [ ]:
fitted1 = fit1.fittedvalues
print(fitted1)

In [ ]:
ax = ts.plot(marker='o', color='black', figsize=(20,8))

fitted1.plot(marker='o', ax=ax, color='blue');

###เราสามารถกำหนดค่า $\alpha$ เองได้

In [ ]:
from statsmodels.tsa.api import SimpleExpSmoothing

fit2 = SimpleExpSmoothing(ts).fit(smoothing_level=0.01,optimized=False)
fitted2 = fit2.fittedvalues

fit3 = SimpleExpSmoothing(ts).fit(smoothing_level=0.9,optimized=False)
fitted3 = fit3.fittedvalues

ax = ts.plot(marker='o', color='black', figsize=(20,8))

fitted1.plot(marker='o', ax=ax, color='blue', legend = True, label = r'$\alpha=%s$'%fit1.model.params['smoothing_level'])
fitted2.plot(marker='o', ax=ax, color='red', legend = True, label = r'$\alpha=0.01$')
fitted3.plot(marker='o', ax=ax, color='green', legend = True, label = r'$\alpha=0.9$')
plt.show()

###ปฏิบัติการครั้งที่ 9 
1. สร้างโมเดล exponential smoothing สองตัวดังต่อไปนี้  
Model 1: โมเดล exponential smoothing ที่สร้างจากข้อมูลที่เรามีอยู่ (เรียกว่า $y_t$)  
Model 2: สร้างโดยเริ่มจากการแยกส่วนประกอบ $y_t=S_t+T_t+R_t$ แล้วทำ exponential smoothing บน $T_t+R_t$ แล้วนำค่าทำนายที่ได้ไปบวกกับ $S_t$ 
2. คำนวณ RMSE ของค่าทำนายจาก Model 1 และ Model 2 แล้วระบุว่าโมเดลไหนเหมาะสมกว่า
3. แสดงแผนภาพของ 1. ข้อมูล elecequip 2. ค่าทำนายจาก Model 1 3. ค่าทำนายจาก Model 2

###เนื่องจาก $T_t$ และ $R_t$ มีข้อมูลที่หายไปจากการทำ 12-MA จึงต้องเอาข้อมูล 6 เดือนแรกและ 6 เดือนสุดท้ายออกไป

In [ ]:
ts_new = ts[6:-6] #ใช้ตอนคำนวณ RMSE

result_add = seasonal_decompose(ts, model='additive')

fitted_new = fitted1[6:-6]
resid_new = result_add.resid[6:-6]
trend_new = result_add.trend[6:-6]
seasonal_new = result_add.seasonal[6:-6]

In [ ]:
def rmse(y_true,y_pred):
  # Inputs
  # y_true: actual value, y_pred: predicted values
  # Output: RMSE between y_true and y_pred
  return np.sqrt(np.mean(np.square(y_true-y_pred)))

In [ ]:
fit = SimpleExpSmoothing(TREND + RESIDUAL).fit()
fitted = fit.fittedvalues

In [ ]:
print(rmse(ts_new,......))

In [ ]:
ax = ts_new.plot(marker='o', color='black', figsize=(20,8))